In [9]:
pip install sec-api



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:

# get your free API key at https://sec-api.io
api_key = "fb0b75eccdcf2997132c668d40782f6d6fbf7d1efc43d18ecbbab4bf5b117c16"


from sec_api import XbrlApi

xbrlApi = XbrlApi(api_key)
# URL of Google's 10-K filings
url_10k = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000045/goog-20230331.htm'
xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k)


In [10]:
import pandas as pd 

# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 


income_statement_google = get_income_statement(xbrl_json)


In [11]:
# Google's 10Ks of the last 5 years, 2018 to 2023, with data from 2016 to 2023
url_10k_2018 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/goog10-kq42018.htm'
url_10k_2019 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204420000008/goog10-k2019.htm'
url_10k_2020 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204421000010/goog-20201231.htm'
url_10k_2021 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204422000019/goog-20211231.htm'
url_10k_2022 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
url_10k_2023 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000045/goog-20230331.htm'

xbrl_json_2018 = xbrlApi.xbrl_to_json(htm_url=url_10k_2018)
xbrl_json_2019 = xbrlApi.xbrl_to_json(htm_url=url_10k_2019)
xbrl_json_2020 = xbrlApi.xbrl_to_json(htm_url=url_10k_2020)
xbrl_json_2021 = xbrlApi.xbrl_to_json(htm_url=url_10k_2021)
xbrl_json_2022 = xbrlApi.xbrl_to_json(htm_url=url_10k_2022)
xbrl_json_2023 = xbrlApi.xbrl_to_json(htm_url=url_10k_2023)

In [12]:
# Fix naming inconsistency for 2020
revenues_2020 = xbrl_json_2020['StatementsOfIncome'].get('Revenues')
if revenues_2020:
    xbrl_json_2020['StatementsOfIncome']['RevenueFromContractWithCustomerExcludingAssessedTax'] = revenues_2020
    del xbrl_json_2020['StatementsOfIncome']['Revenues']

# Fix naming inconsistency for 2021
revenues_2021 = xbrl_json_2021['StatementsOfIncome'].get('Revenues')
if revenues_2021:
    xbrl_json_2021['StatementsOfIncome']['RevenueFromContractWithCustomerExcludingAssessedTax'] = revenues_2021
    del xbrl_json_2021['StatementsOfIncome']['Revenues']

# Fix naming inconsistency for 2022
revenues_2022 = xbrl_json_2022['StatementsOfIncome'].get('Revenues')
if revenues_2022:
    xbrl_json_2022['StatementsOfIncome']['RevenueFromContractWithCustomerExcludingAssessedTax'] = revenues_2022
    del xbrl_json_2022['StatementsOfIncome']['Revenues']


# Fix naming inconsistency for 2023
revenues_2023 = xbrl_json_2023['StatementsOfIncome'].get('Revenues')
if revenues_2023:
    xbrl_json_2023['StatementsOfIncome']['RevenueFromContractWithCustomerExcludingAssessedTax'] = revenues_2023
    del xbrl_json_2023['StatementsOfIncome']['Revenues']

In [13]:
income_statement_2018 = get_income_statement(xbrl_json_2018)
income_statement_2019 = get_income_statement(xbrl_json_2019)
income_statement_2020 = get_income_statement(xbrl_json_2020)
income_statement_2021 = get_income_statement(xbrl_json_2021)
income_statement_2022 = get_income_statement(xbrl_json_2022)
income_statement_2023 = get_income_statement(xbrl_json_2023)

In [14]:
income_statements_merged = pd.concat([income_statement_2018, 
                                      income_statement_2019, 
                                      income_statement_2020, 
                                      income_statement_2021, 
                                      income_statement_2022,
                                      income_statement_2023], axis=0, sort=False)

# sort & reset the index of the merged dataframe
income_statements_merged = income_statements_merged.sort_index().reset_index()

# convert cells to float
income_statements_merged = income_statements_merged.applymap(lambda x: pd.to_numeric(x, errors='ignore'))


In [16]:
income_statements = income_statements_merged.groupby('index').max()

# reindex the merged dataframe using the index of the first dataframe
income_statements = income_statements.reindex(income_statement_2019.index)

# loop over the columns
for col in income_statements.columns[1:]:
    # extract start and end dates from the column label
    splitted = col.split('-')
    start = '-'.join(splitted[:3])
    end = '-'.join(splitted[3:])

    # convert start and end dates to datetime objects
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)

    # calculate the duration between start and end dates
    duration = (end_date - start_date).days / 360

    # drop the column if duration is less than a year
    if duration < 1:
        income_statements.drop(columns=[col], inplace=True)

# convert datatype of cells to readable format, e.g. "2.235460e+11" becomes "223546000000"
income_statements = income_statements.apply(lambda row: pd.to_numeric(row, errors='coerce', downcast='integer').astype(str), axis=1) 


print("Income statements from Google's 10-K filings (2016 to 2022) as dataframe")
print('------------------------------------------------------------------------')
income_statements

Income statements from Google's 10-K filings (2016 to 2022) as dataframe
------------------------------------------------------------------------


/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:383: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:383: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:383: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:383: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:383: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:383: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
/usr/local

,2016-01-01-2016-12-31,2017-01-01-2017-12-31,2018-01-01-2018-12-31,2019-01-01-2019-12-31,2020-01-01-2020-12-31,2021-01-01-2021-12-31,2022-01-01-2022-12-31
RevenueFromContractWithCustomerExcludingAssessedTax,90272000000,110855000000,136819000000,161857000000,182527000000,257637000000,282836000000
CostOfRevenue,35138000000,45583000000,59549000000,71896000000,84732000000,110939000000,126203000000
ResearchAndDevelopmentExpense,13948000000,16625000000,21419000000,26018000000,27573000000,31562000000,39500000000
SellingAndMarketingExpense,10485000000,12893000000,16333000000,18464000000,17946000000,22912000000,26567000000
GeneralAndAdministrativeExpense,6985000000,6872000000,8126000000,9551000000,11052000000,13510000000,15724000000
LossContingencyLossInPeriod,0.0,2736000000.0,5071000000.0,1697000000.0,0.0,0.0,nan
CostsAndExpenses,66556000000,84709000000,110498000000,127626000000,141303000000,178923000000,207994000000
OperatingIncomeLoss,23716000000,26178000000,27524000000,34231000000,41224000000,78714000000,74842000000
NonoperatingIncomeExpense,434000000,1047000000,8592000000,5394000000,6858000000,12020000000,-3514000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments,24150000000.0,27193000000.0,34913000000.0,39625000000.0,48082000000.0,nan,nan
